# Differential erosion through a dipping dyke

This notebook reproduces the [dipping dyke example](https://fastscape-lem.github.io/fastscapelib-fortran/#_dippingdyke_f90) provided in the fastscapelib-fortran library. It shows how to create custom processes for modelling differential erosion due to some structural (e.g., rock strengh) heterogeneity.

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
import fastscape

%matplotlib inline

In [ ]:
print('xarray-simlab version: ', xs.__version__)
print('fastscape version: ', fastscape.__version__)

## Import and inspect the model

We use the basic model available in [fastscape](https://fastscape.readthedocs.io/en/latest/).

In [ ]:
from fastscape.models import basic_model

Let's create a new process class to model the effect of the dyke on the efficiency of erosion processes.

In [ ]:
from fastscape.processes import (LinearDiffusion, StreamPowerChannel,
                                 TotalErosion, UniformRectilinearGrid2D)


@xs.process
class DippingDyke:
    """Mimics the effect on erosion rates of a dyke dipping at
    a given angle, that is buried beneath the landscape and that is
    progressively exhumed by erosion.
    
    """
    x_position = xs.variable(description='initial x-position of exposed dyke')
    width = xs.variable(description='dyke fixed width')
    angle = xs.variable(description='dyke dipping angle in degrees')
    
    grid_shape = xs.foreign(UniformRectilinearGrid2D, 'shape')
    x = xs.foreign(UniformRectilinearGrid2D, 'x')
    
    etot = xs.foreign(TotalErosion, 'cumulative_height')
    
    k_coef = xs.foreign(StreamPowerChannel, 'k_coef', intent='out')
    diffusivity = xs.foreign(LinearDiffusion, 'diffusivity', intent='out')
    
    def run_step(self):
        cotg = 1. / np.tan(np.radians(self.angle))
        
        dyke_pos = self.x - self.x_position - self.etot * cotg
        
        in_dyke = (dyke_pos - self.width) * (dyke_pos + self.width) <= 0
        
        self.k_coef = np.where(in_dyke, 1e-5, 2e-5)
        self.diffusivity = np.where(in_dyke, 0.05, 0.1)


In [ ]:
model = basic_model.update_processes({'dyke': DippingDyke})

model

In [ ]:
model.visualize(show_inputs=True)

## Model setup

**Note**: One important difference between this model and the example shown in the fastscapelib-fortran library is that linear diffusion is here computed using the same topographic surface than the one used for computing stream power channel erosion. In fastscapelib-fortran, diffusion is always computed after applying channel erosion on the topographic surface, which makes it more "tolerant" to large time steps.

While it is possible to customize `basic_model` to mimic the behavior of fastscapelib-fortran (e.g., by replacing the diffusion process by another (sub)class), we keep the "apply-combine" approach here for erosion processes for more flexibility. Even if those processes are applied sequentially, setting large time steps has a significant impact on the solution as the latter is is only partially implicit (when considering the processes all together).

In [ ]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'time': np.arange(0, 4e7 + 2e4, 4e4),
        'out': np.arange(0, 4e7 + 4e5, 4e5),
    },
    master_clock='time',
    input_vars={
        'grid__shape': ('shape_yx', [201, 201]),
        'grid__length': ('shape_yx', [1e5, 1e5]),
        'boundary__status': ('border', ['fixed_value', 'fixed_value', 'fixed_value', 'fixed_value']),
        'dyke': {
            'x_position': 1e4,
            'width': 2e3,
            'angle': 30.
        },
        'uplift__rate': 1e-3,
        'spl': {
            'area_exp': 0.4,
            'slope_exp': 1.
        }
    },
    output_vars={
        'out': ['topography__elevation',
                'terrain__slope'],
        None: ['boundary__border',
               'grid__x',
               'grid__y',
               'grid__spacing'],
    }
)

in_ds

## Run the model

This should take 20-30 seconds to run.

In [ ]:
out_ds = (in_ds.xsimlab.run(model=model)
               .set_index(x='grid__x', y='grid__y',
                          border='boundary__border'))

In [ ]:
out_ds

## Plot the outputs

Plot the local topographic gradient (slope) and the swath profile of elevations averaged along the y-axis.

In [ ]:
import hvplot.xarray
import holoviews as hv
from xshade import hillshade


slope_plot = out_ds.terrain__slope.hvplot.image(
    x='x', y='y', clim=(0., 40.),
    width=550, height=450,
    cmap=plt.cm.plasma, groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap=plt.cm.gray, alpha=0.6,
    colorbar=False, hover=False, groupby='out'
)

profile_plot = out_ds.topography__elevation.mean('y').hvplot.line(
    width=550, height=250, ylim=(0, 2000),
    groupby='out', legend='top_left',
)

hv.Layout((slope_plot * hillshade_plot) + profile_plot).cols(1)